<a href="https://colab.research.google.com/github/joseaureliok/soulcode-martech-bootcamp-ad/blob/notebooks_aulas/aulas-resumos/04_resumo_AD2_an%C3%A1lise_de_dados_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PYSPARK PARA ANÁLISE DE DADOS**

Este notebook foi elaborado no Google Colaboratory e posteriormente ajustado pelo VS Code, em forma de compilação dos principais pontos do apresentados sobre o tema durante o curso.

#### Bootcamp Martech em Análise de Dados - AD2
##### SoulCode Academy
##### Professores:
                Franciane Rodrigues
                Douglas Ribeiro
                Jonatas Carneiro
##### Acadêmico:
                José Aurelio Kovalczuk de Oliveira

## **Configuração do PySpark**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável
spark

## **Bibliotecas**

In [ ]:
# instalação de Bibiliotecas
!pip install pandera
!pip install gcsfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
# importação de Bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandera as pa
import seaborn as sns
import warnings

import pyspark.sql.functions as f

from statistics import mode

from pyspark.sql.types import *
from google.cloud import storage
from google.colab import drive
from pandas import DataFrame

In [ ]:
# retirada de alertas
warnings.filterwarnings("ignore")

## **Conjunto de dados**

In [ ]:
estrutura = StructType([\
                     StructField('time',StringType(),True), \
                     StructField('pontos',IntegerType(),True), \
                     StructField('jogos',IntegerType(),True), \
                     StructField('vitória',IntegerType(),True), \
                     StructField('empate', IntegerType(), True), \
                     StructField('derrota', IntegerType(), True),\
                     StructField('gols_pro', IntegerType(), True),\
                     StructField('gols_contra', IntegerType(), True),\
                     StructField('gols_saldo', IntegerType(), True),\
                     StructField('aproveitamento', DoubleType(), True)
                    ])

dados = [('Botafogo', 36,14,12,0,2,24,7,17,85.0),
         ('Flamengo', 26	,14	,8	,2	,4	,27	,18	,9	,61.0),
         ('Grêmio',26	,14	,8	,2	,4	,24	,20	,4	,61.0),
         ('Fluminense',24	,14	,7	,3	,4	,21	,14	,7	,57.0),
         ('Palmeiras',24	,14	,6	,6	,2	,25	,13	,12	,57.0),
         ('Bragantino',24	,14	,6	,6	,2	,22	,14	,8	,57.0),
         ('Fortaleza',23	,14	,6	,5	,3	,16	,11	,5	,54.0),
         ('São Paulo',22,14	,6	,4	,4	,18	,13	,5	,52.0),
         ('Cruzeiro',21	,14	,6	,3	,5	,16	,11	,5	,50.0),
         ('Internacional',21	,14	,6	,3	,5	,13	,16	,-3	,50.0),
         ('Athletico-PR',20	,14	,6	,2	,6	,18	,18	,0	,47.0),
         ('Atlético-MG',20	,14	,5	,5	,4	,18	,14	,4	,47.0),
         ('Santos',16	,14	,4	,4	,6	,15	,19	,-4	,38.0),
         ('Cuiabá',16	,14	,4	,4	,6	,13	,17	,-4	,38.0),
         ('Corinthians',15	,14	,4	,3	,7	,12	,17	,-5	,35.0),
         ('Bahia',13	,14	,3	,4	,7	,15	,21	,-6	,30.0),
         ('Goiás',11	,14	,3	,2	,9	,14	,26	,-12	,26.0),
         ('Coritiba',10	,14	,2	,4	,8	,14	,28	,-14	,23.0),
         ('América-MG',9	,14	,2	,3	,9	,11	,23	,-12	,21.0),
         ('Vasco', 9	,14	,2	,3	,9	,16	,32	,-16	,21.0)
]

## **Criando dataframe com PySpark**

In [ ]:
# criando dataframe com dados e estrutura criada
dfx = spark.createDataFrame(data=dados, schema=estrutura)

In [ ]:
# exibindo dataframe com show
dfx.show()

+-------------+------+-----+-------+------+-------+--------+-----------+----------+--------------+
|         time|pontos|jogos|vitória|empate|derrota|gols_pro|gols_contra|gols_saldo|aproveitamento|
+-------------+------+-----+-------+------+-------+--------+-----------+----------+--------------+
|     Botafogo|    36|   14|     12|     0|      2|      24|          7|        17|          85.0|
|     Flamengo|    26|   14|      8|     2|      4|      27|         18|         9|          61.0|
|       Grêmio|    26|   14|      8|     2|      4|      24|         20|         4|          61.0|
|   Fluminense|    24|   14|      7|     3|      4|      21|         14|         7|          57.0|
|    Palmeiras|    24|   14|      6|     6|      2|      25|         13|        12|          57.0|
|   Bragantino|    24|   14|      6|     6|      2|      22|         14|         8|          57.0|
|    Fortaleza|    23|   14|      6|     5|      3|      16|         11|         5|          54.0|
|    São P

In [ ]:
# exibindo Schema
dfx.printSchema()

root
 |-- time: string (nullable = true)
 |-- pontos: integer (nullable = true)
 |-- jogos: integer (nullable = true)
 |-- vitória: integer (nullable = true)
 |-- empate: integer (nullable = true)
 |-- derrota: integer (nullable = true)
 |-- gols_pro: integer (nullable = true)
 |-- gols_contra: integer (nullable = true)
 |-- gols_saldo: integer (nullable = true)
 |-- aproveitamento: double (nullable = true)



## **Describe | Summary**

In [ ]:
# exibindo medidas estatística com summary
dfx.summary()

summary,time,pontos,jogos,vitória,empate,derrota,gols_pro,gols_contra,gols_saldo,aproveitamento
count,20,20,20,20,20,20,20,20,20,20
mean,null,19.3,14.0,5.3,3.4,5.3,17.6,17.6,0.0,45.5
stddev,null,6.92136281074,0.0,2.4516374764204993,1.465390194130093,2.3192557789991715,4.706099066217435,6.150738081512529,9.107544587925586,16.458880181898934
min,América-MG,9,14,2,0,2,11,7,-16,21.0
25%,null,13,14,3,2,4,14,13,-6,30.0
50%,null,20,14,6,3,5,16,17,0,47.0
75%,null,24,14,6,4,7,21,20,5,57.0
max,Vasco,36,14,12,6,9,27,32,17,85.0


In [ ]:
# exibindo descrição com estatísticas para o dataframe com describe
dfx.describe()

summary,time,pontos,jogos,vitória,empate,derrota,gols_pro,gols_contra,gols_saldo,aproveitamento
count,20,20,20,20,20,20,20,20,20,20
mean,null,19.3,14.0,5.3,3.4,5.3,17.6,17.6,0.0,45.5
stddev,null,6.92136281074,0.0,2.4516374764204993,1.465390194130093,2.3192557789991715,4.706099066217435,6.150738081512529,9.107544587925586,16.458880181898934
min,América-MG,9,14,2,0,2,11,7,-16,21.0
max,Vasco,36,14,12,6,9,27,32,17,85.0


In [ ]:
# passando os dados em um novo dataframe
edescritiva = dfx.describe()

## **Lista desvio-padrão**

In [ ]:
# extraindo linha com desvio padrão do dataframe
row_sd = edescritiva.select('*').where( f.col('summary')== 'stddev').collect()

In [ ]:
# exibindo linha no formato de lista coletada
print(row_sd)

[Row(summary='stddev', time=None, pontos='6.92136281074', jogos='0.0', vitória='2.4516374764204993', empate='1.465390194130093', derrota='2.3192557789991715', gols_pro='4.706099066217435', gols_contra='6.150738081512529', gols_saldo='9.107544587925586', aproveitamento='16.458880181898934')]


In [ ]:
# selecionando valores para lista sd
sd = list(row_sd[0][2:])

In [ ]:
# exibindo valores na lista sd
print(sd)
# exibindo tipo de dados da lista sd
type(sd)

['6.92136281074', '0.0', '2.4516374764204993', '1.465390194130093', '2.3192557789991715', '4.706099066217435', '6.150738081512529', '9.107544587925586', '16.458880181898934']


list

## **Lista variância**

In [ ]:
# criando lista vazia de nome var
var = []
# laço FOR
for n in sd:
  # verificação do tipo de dado
  if type(n) != type(float):
    try:
      # conversão em float
      n = float(n)
      var.append(n**2)
    except:
      # sinalização do erro
      print(f'Erro em {n}!')
  else:
     var.append(n**2)

In [ ]:
# exibição da lista com a variância
print(var)
# exibição da variância
for n in var:
  print( f'{n}: {type(n)}')

[47.90526315789471, 0.0, 6.010526315789474, 2.147368421052632, 5.378947368421054, 22.147368421052615, 37.83157894736842, 82.94736842105264, 270.8947368421055]
47.90526315789471: <class 'float'>
0.0: <class 'float'>
6.010526315789474: <class 'float'>
2.147368421052632: <class 'float'>
5.378947368421054: <class 'float'>
22.147368421052615: <class 'float'>
37.83157894736842: <class 'float'>
82.94736842105264: <class 'float'>
270.8947368421055: <class 'float'>


## **Criação do dataframe**

In [ ]:
# exibição da lista desvio padrão
print(sd)
# exibição da lista variância
print(var)

['6.92136281074', '0.0', '2.4516374764204993', '1.465390194130093', '2.3192557789991715', '4.706099066217435', '6.150738081512529', '9.107544587925586', '16.458880181898934']
[47.90526315789471, 0.0, 6.010526315789474, 2.147368421052632, 5.378947368421054, 22.147368421052615, 37.83157894736842, 82.94736842105264, 270.8947368421055]


In [ ]:
# extração de colunas do dataframe inicial
xcols = dfx.columns[1:]

In [ ]:
# criação do dataframe desvio-padrão
df_sd = spark.createDataFrame([sd], xcols)

In [ ]:
# exibição do dataframe
df_sd.show()

+-------------+-----+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+
|       pontos|jogos|           vitória|           empate|           derrota|         gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+-------------+-----+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+
|6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715|4.706099066217435|6.150738081512529|9.107544587925586|16.458880181898934|
+-------------+-----+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+



In [ ]:
# criação do dataframe variância
df_var = spark.createDataFrame([var], xcols)

In [ ]:
# exibição do dataframe
df_var.show()

+-----------------+-----+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|           pontos|jogos|          vitória|           empate|          derrota|          gols_pro|      gols_contra|       gols_saldo|   aproveitamento|
+-----------------+-----+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|47.90526315789471|  0.0|6.010526315789474|2.147368421052632|5.378947368421054|22.147368421052615|37.83157894736842|82.94736842105264|270.8947368421055|
+-----------------+-----+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+



## **Agrupamento de Dataframes**

In [ ]:
# inserindo sumário nos dataframes antes da união
df_sd = df_sd.withColumn( 'summary', f.expr("'desvio-padrão'"))
df_var = df_var.withColumn( 'summary', f.expr("'variância'"))

# inserindo summary na lista de colunas
xcols.insert(0, 'summary')

# reordenando dataframes
df_sd = df_sd.select(xcols)
df_var = df_var.select(xcols)

In [ ]:
# união dataframes
df_sd_var = df_sd.union(df_var)

In [ ]:
# exibição do dataframe desvio e variancia
df_sd_var.show()

+-------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|      summary|           pontos|jogos|           vitória|           empate|           derrota|          gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+-------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|desvio-padrão|    6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715| 4.706099066217435|6.150738081512529|9.107544587925586|16.458880181898934|
|    variância|47.90526315789471|  0.0| 6.010526315789474|2.147368421052632| 5.378947368421054|22.147368421052615|37.83157894736842|82.94736842105264| 270.8947368421055|
+-------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+

## **Dataframe Medidas**

In [ ]:
# criação de dataframe com medidas estatísticas
medidas = dfx.summary()

In [ ]:
# exclusão de coluna com medida qualitativa
medidas = medidas.drop('time')

In [ ]:
# adição da medida de variância para cada atributo
medidas = medidas.union(df_var)

In [ ]:
# exibição da tabela
medidas.show()

+---------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|  summary|           pontos|jogos|           vitória|           empate|           derrota|          gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+---------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|    count|               20|   20|                20|               20|                20|                20|               20|               20|                20|
|     mean|             19.3| 14.0|               5.3|              3.4|               5.3|              17.6|             17.6|              0.0|              45.5|
|   stddev|    6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715| 4.706099066217435|6.150738081512529|9.107544587925586|16.458880181898934|
|   

## **Moda**

In [ ]:
# calcular a moda de todas as colunas
df = dfx
columns = df.columns
mode_values = {}

for column in columns:
    mode_df = df.groupBy(column).count().sort(f.col("count").desc()).limit(1)
    mode_value = mode_df.select(column).first()[0]
    mode_values[column] = mode_value

In [ ]:
# criação df
df_mode = spark.createDataFrame([mode_values])

In [ ]:
# ajuste lista
dfz = medidas.drop('summary')
zcols = dfz.columns

In [ ]:
# ordenação e seleção de colunas
df_mode = df_mode.select(zcols)

In [ ]:
# adição de coluna e valor
dfm = df_mode.withColumn('summary', f.expr("'moda'"))

In [ ]:
# reordenação e inserção de valor na lista de colunas
zcols.insert(0, "summary")

In [ ]:
# seleção e ordenação de colunas
dfm = dfm.select(zcols)

In [ ]:
# adição da moda ao df de medidas estatísticas
estatisticas = medidas.union(dfm)

In [ ]:
# exibição do dataframe de medidas estatísticas
estatisticas.show()

+---------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|  summary|           pontos|jogos|           vitória|           empate|           derrota|          gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+---------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|    count|               20|   20|                20|               20|                20|                20|               20|               20|                20|
|     mean|             19.3| 14.0|               5.3|              3.4|               5.3|              17.6|             17.6|              0.0|              45.5|
|   stddev|    6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715| 4.706099066217435|6.150738081512529|9.107544587925586|16.458880181898934|
|   

## **Estatísticas descritivas**

In [ ]:
# seleção de listas para os cálculos
q1 = list(((estatisticas.select('*').where( f.col('summary')== '25%').collect())[0][1:]))
q3 = list((estatisticas.select('*').where( f.col('summary')== '75%').collect())[0][1:])
min = list((estatisticas.select('*').where( f.col('summary')== 'min').collect())[0][1:])
max = list((estatisticas.select('*').where( f.col('summary')== 'max').collect())[0][1:])

In [ ]:
# criação de listas vazias para alocar as novas medidas
at = []
aiq = []
ls = []
li = []

In [ ]:
# calculando numero de colunas com valores quantitativos
e_len = len(estatisticas.columns[1:])

# cálculos de novas medidas em um laço FOR
for i in range(e_len):
  at.append((float(max[i]))-(float(min[i])))
  aiq.append((float(q3[i]))-(float(q1[i])))
  ls.append((float(q3[i]))+(1.5*(float(aiq[i]))))
  li.append((float(q1[i]))-(1.5*(float(aiq[i]))))

In [ ]:
# inserindo index nas listas
at.insert(0, 'amplitude total')
aiq.insert(0, 'amplitude interquartílica')
ls.insert(0, 'limite superior')
li.insert(0, 'limite inferior')

In [ ]:
# passando valores com withColumn
df_at = spark.createDataFrame([at], zcols)
df_aiq = spark.createDataFrame([aiq], zcols)
df_ls = spark.createDataFrame([ls], zcols)
df_li = spark.createDataFrame([li], zcols)

In [ ]:
# união dos dataframes
estatisticasFull = estatisticas \
                                .union(df_at) \
                                .union(df_aiq) \
                                .union(df_li) \
                                .union(df_ls)

In [ ]:
# exibição de estatísticas completas
estatisticasFull.show()

+--------------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|             summary|           pontos|jogos|           vitória|           empate|           derrota|          gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+--------------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|               count|               20|   20|                20|               20|                20|                20|               20|               20|                20|
|                mean|             19.3| 14.0|               5.3|              3.4|               5.3|              17.6|             17.6|              0.0|              45.5|
|              stddev|    6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715| 4.7060990662

In [ ]:
# ajustando nomes dos índices (rows)
estatisticasFull = estatisticasFull.na.replace(["count", "mean", "stddev", "25%", "50%", "75%"], ["quantidade", "média", "desvio padrão", "primeiro quartil", "mediana", "terceiro quartil"])

In [ ]:
# ajustando nome dos atributos (columns)
estatisticasFull = estatisticasFull.withColumnRenamed("summary", "descrição")

In [ ]:
# exibindo alterações
estatisticasFull.show()

+--------------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|           descrição|           pontos|jogos|           vitória|           empate|           derrota|          gols_pro|      gols_contra|       gols_saldo|    aproveitamento|
+--------------------+-----------------+-----+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+
|          quantidade|               20|   20|                20|               20|                20|                20|               20|               20|                20|
|               média|             19.3| 14.0|               5.3|              3.4|               5.3|              17.6|             17.6|              0.0|              45.5|
|       desvio padrão|    6.92136281074|  0.0|2.4516374764204993|1.465390194130093|2.3192557789991715| 4.7060990662

### *Alternativas*

In [ ]:
# obtendo quartis com a função do PySpark approxQuantile()
quartis = dfx.approxQuantile(['pontos', 'vitória', 'gols_saldo'], [0.25, 0.50, 0.75], 0)
quartis

[[13.0, 20.0, 24.0], [3.0, 6.0, 6.0], [-6.0, 0.0, 5.0]]

In [ ]:
# extraindo quartis, conforme posiçao e ordem passadas
primeiro_quartil = [quartis[0][0], quartis[0][1], quartis[0][2]]
mediana = [quartis[1][0], quartis[1][1], quartis[1][2]]
terceiro_quartil = [quartis[2][0], quartis[2][1], quartis[2][2]]

In [ ]:
# conferindo valores
estatisticasFull.select('summary', 'pontos', 'vitória', 'gols_saldo').where( f.col ('summary').isin(['25%', '50%', '75%']))

# exibição
print( f'Q1:{primeiro_quartil}\nQ2:{mediana}\nQ3:{terceiro_quartil}')

summary,pontos,vitória,gols_saldo
25%,13,3,-6
50%,20,6,0
75%,24,6,5


In [ ]:
# lista para usar laço
nlist = [primeiro_quartil, mediana, terceiro_quartil]

# laço FOR para percorrer lista com percentis
for n in nlist:
  # laço FOR para transformar valores no tipo float
  n = [float(i) for i in n]

print( f'Q1:{type(primeiro_quartil[0]), primeiro_quartil}\nQ2:{type(mediana[0]), mediana}\nQ3:{type(terceiro_quartil[0]), terceiro_quartil}')

Q1:(<class 'float'>, [13.0, 20.0, 24.0])
Q2:(<class 'float'>, [3.0, 6.0, 6.0])
Q3:(<class 'float'>, [-6.0, 0.0, 5.0])


In [ ]:
df_py = dfx
#Lista todas as Colunas do dataframe
todas_colunas = [coluna for coluna, tipo in df_py.dtypes ]

#Lista todas as Colunas do dataframe
colunas_numericas = [coluna for coluna, tipo in df_py.dtypes if tipo in ['double', 'float', 'decimal', 'integer', 'long']]

# Calculando os quartis de todos os atributos quantitativos
quartis = df_py.approxQuantile(colunas_numericas, [0.25,0.5,0.75], 0)

quartis

# **FIM**

Estes foram os principais fundamentos aboradados durante o curso!